# **Volumetric Segmentation of Drosophila Longitudinal muscles**
---

<font size = 4>Indirect flight muscles (IFMs) in adult Drosophila provide the key power stroke for wing beating. They also serve as a valuable model for studying muscle development. Such analyses are impeded by conventional histological preparations and imaging techniques that limit exact morphometry of flight muscles. In this tutorial, microCT scanning is employed on a tissue preparation that retains muscle morphology under homeostatic conditions. Focusing on a subset of IFMs called the dorsal longitudinal muscles (DLMs), it is found that DLM volumes increase with age, partially due to the increased separation between myofibrillar fascicles, in a sex-dependentmanner.The authors have uncovered and quantified asymmetry in the size of these muscles on either side of the longitudinal midline.Measurements of this resolution and scalemake substantive studies that test the connection between form and function possible.


<font size = 4> **In this tutorial, you will learn how to predict the Drosophila longitudinal muscles instance masks on the CT Fly dataset released by Chaturvedi, et al. in 2019.**



<font size = 4>**The Original code** is freely available in GitHub:
https://github.com/aayushk614/pytorch_connectomics/tree/master/projects/microCT



# **1. Initialise the Colab session**




---






## **1.1. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive. 

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

#@markdown * Click on the URL. 

#@markdown * Sign in your Google Account. 

#@markdown * Copy the authorization code. 

#@markdown * Enter the authorization code. 

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive". 

# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **2. Install the packages and dependencies**
---


In [ ]:
from threedunet import unet_residual_3d

import imageio
import h5py

import torch
import os,datetime
import torch
import torch.nn as nn
import numpy as np
from torchsummary import summary


def read_h5(filename, dataset=''):
    fid = h5py.File(filename, 'r')
    if dataset == '':
        dataset = list(fid)[0]
    return np.array(fid[dataset])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model= unet_residual_3d(in_channel=1, out_channel=13).to(device)
model = nn.DataParallel(model, device_ids=range(1))
microct_model = model.to(device)

In [ ]:
summary(microct_model,(1,112,112,112))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
  ReplicationPad3d-1     [-1, 1, 112, 116, 116]               0
            Conv3d-2    [-1, 28, 112, 112, 112]             728
SynchronizedBatchNorm3d-3    [-1, 28, 112, 112, 112]              56
               ELU-4    [-1, 28, 112, 112, 112]               0
  ReplicationPad3d-5    [-1, 28, 112, 114, 114]               0
            Conv3d-6    [-1, 28, 112, 112, 112]           7,084
SynchronizedBatchNorm3d-7    [-1, 28, 112, 112, 112]              56
               ELU-8    [-1, 28, 112, 112, 112]               0
  ReplicationPad3d-9    [-1, 28, 112, 114, 114]               0
           Conv3d-10    [-1, 28, 112, 112, 112]           7,084
SynchronizedBatchNorm3d-11    [-1, 28, 112, 112, 112]              56
              ELU-12    [-1, 28, 112, 112, 112]               0
 ReplicationPad3d-13    [-1, 28, 112, 114, 114]               0
           Conv3d-14   

# **3. Select and load your model checkpoint and testing volume**
---

## **3.1. Setting model checkpoint or weights**
---
<font size = 4>  Here, you can set the the path to a pre-trained model from which the weights can be extracted and used for carrying out the predictions. **This pre-trained model needs to be a 3D Unet Residual model**. 

In [ ]:
# load pre-trained model
checkpoint = 'checkpoint_90000.pth.tar'

In [ ]:

print('Load pretrained checkpoint: ', checkpoint)
checkpoint = torch.load(checkpoint)
print('checkpoints: ', checkpoint.keys())

# update model weights
if 'state_dict' in checkpoint.keys():
    pretrained_dict = checkpoint['state_dict']
    model_dict = microct_model.state_dict() # nn.DataParallel
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict 
    model_dict.update(pretrained_dict)    
    # 3. load the new state dict
    microct_model.load_state_dict(model_dict) # nn.DataParallel
    
    print("new state dict loaded ")
    


## **3.2. Select the test Volume**
---


In [ ]:
Dim = 'MvsFCSfemaleday2-4-DownSamp_im.h5'

Prepare the training data into a suitable format for testing.


In [ ]:
image_volume = read_h5(Dim)   #reading CT volume 
print(image_volume.shape)
vol = image_volume

volume = torch.from_numpy(vol).to(device, dtype=torch.float)
volume = volume.unsqueeze(0)

volume = volume.unsqueeze(0)

print(volume.shape)

## **6.1. Generate prediction(s) from testing dataset**
---

<font size = 4>The current trained model (from section 3.1) can now be used to process testing volumes. 





In [ ]:
pred = microct_model(volume)

Check if the predictions are in correct format or not.

In [ ]:
print("Shape of pred after test", pred.shape)
pred = pred.squeeze(0)
print("Shape of pred after test", pred.shape)
pred = pred.cpu()
arr1 = np.argmax(pred.detach().numpy(),axis=0).astype(np.uint16)
print("shape of Predictions after argmax() function ", arr1.shape)

## **6.2. Save and Download your predictions**
---

<font size = 4> 

In [ ]:
hf1 = h5py.File('MvsFCSfemaleday2-4-DownSamp_pred.h5', 'w')
hf1.create_dataset('dataset1', data=arr1)
print("pred volume created and saved" , hf1)
hf1.close()

#**Thank you for using the tutorial!**